# Day 4

## Problem 1

A standard 2D board/word-search type problem

In [1]:
WORD = "XMAS"

# possible directions
directions = [
    # row, column
    (-1, -1),  # top left
    (-1, 0),  # top
    (-1, 1),  # top right
    (0, -1),  # left
    (0, 1),  # right
    (1, -1),  # bottom left
    (1, 0),  # bottom
    (1, 1),  # bottom right
]


def visit(position, partial, direction, board):
    # unpack position
    row, col = position
    row_delta, col_delta = direction

    # base case
    if partial == WORD:
        return 1

    # check out of bounds
    if row < 0 or row >= len(board):
        return 0
    if col < 0 or col >= len(board[0]):
        return 0

    # we can continue looking for the next letter
    looking_for = WORD[len(partial)]

    # check if we found the next letter
    if board[row][col] == looking_for:
        # continue in the same direction
        return visit(
            position=(row + row_delta, col + col_delta),
            partial=partial + looking_for,  # lock in the next letter
            direction=direction,
            board=board,
        )
    else:
        return 0


def p1(board):
    sum = 0
    # for each row
    for i, row in enumerate(board):
        # for each col (and letter)
        for j, letter in enumerate(row):
            # if its the first letter of the word
            if letter == WORD[0]:
                # check all directions for the next letter
                for direction in directions:
                    row_delta, col_delta = direction
                    sum += visit(
                        position=(i + row_delta, j + col_delta),
                        partial=WORD[0],  # we've locked in the first letter
                        direction=direction,
                        board=board,
                    )
    return sum

### Example

In [2]:
board = []
with open("./data/day4_small_input_p1.txt") as f:
    for line in f:
        board.append(line.strip())
p1(board)

4

### Full Problem

In [3]:
board = []
with open("./data/day4_input.txt") as f:
    for line in f:
        board.append(line.strip())
p1(board)

2536

## Problem 2

This problem actually tries to find two `MAS` that  cross - like:

```
M.S
.A.
M.S
```

My first thoughts:

- Search for all `MAS`, and then somehow intersect them to find the crossing ones? May be able to reuse code from above and just write an intersection piece to cut down on matches.
- Another approach is to start mostly fresh and search for an `A`, and then just check its diagonals...

Lets try the first approach since I think the idea is to try and reuse code in advent of code.

In [4]:
WORD = "MAS"
board = []
with open("./data/day4_small_input_p2.txt") as f:
    for line in f:
        board.append(line.strip())
p1(board)

20

`20` is correct for just full `matches` we now need to trim them down to `crosses`.

Actually...instead of just using `p1` and then scanning board again... lets just modify it directly.

The new idea is to instead change our directions of search to only include diagonals. When I find a diagonal, I'll add the position of its `A` to a dict with the number of matches its involved in. Then I'll just check the dict for any positions with a value of `2` and include them in the sum.

In [5]:
from collections import defaultdict


WORD = "MAS"

# possible directions
directions = [
    # row, column
    (-1, -1),  # top left
    (-1, 1),  # top right
    (1, -1),  # bottom left
    (1, 1),  # bottom right
]


def visit_p2(position, partial, direction, board, matches):
    # unpack position
    row, col = position
    row_delta, col_delta = direction

    # base case
    if partial == WORD:
        # needed to debug to work out its two steps back direction
        # print(f"Found {WORD} it ends at: {row-row_delta} and {col-col_delta}")
        # print(f"  Saving A at {row-row_delta-row_delta} and {col-col_delta-col_delta}")
        # location of A:
        loc_A = (row - row_delta - row_delta, col - col_delta - col_delta)
        matches[loc_A] += 1
        return

    # check out of bounds
    if row < 0 or row >= len(board):
        return
    if col < 0 or col >= len(board[0]):
        return

    # we can continue looking for the next letter
    looking_for = WORD[len(partial)]

    # check if we found the next letter
    if board[row][col] == looking_for:
        # continue in the same direction
        visit_p2(
            position=(row + row_delta, col + col_delta),
            partial=partial + looking_for,  # lock in the next letter
            direction=direction,
            board=board,
            matches=matches,
        )
    else:
        return


def p2(board):
    matches = defaultdict(int)  # automatically initializes missing keys to 0

    # for each row
    for i, row in enumerate(board):
        # for each col (and letter)
        for j, letter in enumerate(row):
            # if its the first letter of the word
            if letter == WORD[0]:
                # check all directions for the next letter
                for direction in directions:
                    row_delta, col_delta = direction
                    visit_p2(
                        position=(i + row_delta, j + col_delta),
                        partial=WORD[0],  # we've locked in the first letter
                        direction=direction,
                        board=board,
                        matches=matches,
                    )

    sum = 0
    for k, v in matches.items():
        if v >= 2:
            sum += 1
    return sum

### Example

In [6]:
WORD = "MAS"
board = []
with open("./data/day4_small_input_p2.txt") as f:
    for line in f:
        board.append(line.strip())
p2(board)

9

### Full Problem

In [7]:
WORD = "MAS"
board = []
with open("./data/day4_input.txt") as f:
    for line in f:
        board.append(line.strip())
p2(board)

1875